# CMIP6 Split Data into Variables

**Following steps are included in this script:**

1. Load Data
2. Split and Save New Data Files

This is necessary if you saved all your variables in one data file and want to store them seperatly now. Please drop all redundent dimensions and coordinates before and be carefull as here only the dimensions time, lat, lon and depth are considered.

In [1]:
# ========== Packages ==========
import xarray as xr
import dask
import os

### Functions

In [2]:
def split_and_save(ds_dict): 
    """
    Split and save xarray datasets in a dictionary into separate netCDF files.

    Args:
    ds_dict (dict): A dictionary containing xarray datasets.
   
    Returns:
    None
    """

    for key, ds in ds_dict.items():
        for var in ds:
            # Variable to keep
            variable_to_keep = var
            dimensions_to_keep = {'time', 'lat', 'lon'}
            coordinates_to_keep = {'time', 'lat', 'lon'}

            if any('depth' in ds[var].dims for var in ds.variables):
                dimensions_to_keep.add('depth')
                coordinates_to_keep.add('depth')

            # Create a new dataset with only the desired variable
            ds_var = ds[[variable_to_keep]]

            # Keep only the desired dimensions
            ds_var = ds_var.isel({dim: slice(None) for dim in dimensions_to_keep.intersection(ds_var.dims)})

            # Set the desired coordinates
            coords_to_set = set(ds_var.variables).intersection(coordinates_to_keep)
            ds_var = ds_var.set_coords(list(coords_to_set))

            savepath = f'../../data/CMIP6/{ds_var.experiment_id}/raw/{var}/'
            filename = f'CMIP.{ds_var.source_id}.{ds_var.experiment_id}.{var}.nc'
            nc_out = os.path.join(savepath, filename)
            os.makedirs(savepath, exist_ok=True) 
            if os.path.exists(nc_out):
                 #   inp = input(f"Delete old file {filename} (y/n):")
                 #   if inp.lower() in ["y"]:
                        os.remove(nc_out)
                        print(f"File  with path: {nc_out} removed")
                 #   else:
                 #       filename = "temp_file.nc"
                 #       nc_out = os.path.join(savepath, filename)
                 #       print(f"Filename change to {filename}")
                        
            # Save to netcdf file
            with dask.config.set(scheduler='threads'):
                ds_var.to_netcdf(nc_out)
                print(f"File with path: {nc_out} saved")

### 1. Load Data

In [23]:
# ========= Define period, models and path ==============
experiment_id = 'historical'
source_id = ['SAM0-UNICON'] # 'TaiESM1', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CanESM5', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'UKESM1-0-LL', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'NorESM2-MM'], 
savepath = f'../../data/CMIP6/{experiment_id}/raw/'

# ========= Use Dask to parallelize computations ==========
dask.config.set(scheduler='processes')

# ========= Create a helper function to open the dataset ========
def open_dataset(filename):
    ds = xr.open_dataset(filename)
    return ds

# ========= Create dictionary using a dictionary comprehension and Dask =======
ds_dict, = dask.compute({model: open_dataset(os.path.join(savepath, f'CMIP.{model}.{experiment_id}.nc'))
                        for model in source_id})

In [24]:
# ========= Have a look into the dictionary =======
print(list(ds_dict.keys()))
ds_dict[list(ds_dict.keys())[0]]

['SAM0-UNICON']


<xarray.Dataset>
Dimensions:     (member_id: 1, time: 1980, depth: 15, lat: 192, lon: 288)
Coordinates:
  * time        (time) object 1850-01-16 11:45:00 ... 2014-12-16 12:00:00
  * depth       (depth) float64 0.007101 0.02792 0.06226 ... 12.93 21.33 35.18
  * lat         (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon         (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * member_id   (member_id) object 'r1i1p1f1'
Data variables: (12/15)
    mrsol       (member_id, time, depth, lat, lon) float32 ...
    evspsbl     (member_id, time, lat, lon) float32 ...
    hurs        (member_id, time, lat, lon) float32 ...
    huss        (member_id, time, lat, lon) float32 ...
    pr          (member_id, time, lat, lon) float32 ...
    evspsblsoi  (member_id, time, lat, lon) float32 ...
    ...          ...
    mrro        (member_id, time, lat, lon) float32 ...
    mrros       (member_id, time, lat, lon) float32 ...
    mrso        (member_id, time, lat, lon) float32 ...
    mrsos       (member_id, time, lat, lon) float32 ...
    npp         (member_id, time, lat, lon) float32 ...
    tran        (member_id, time, lat, lon) float32 ...
Attributes: (12/49)
    source:                    SAM0-UNICON (2017): \naerosol: MAM3\natmos: CA...
    initialization_index:      1
    grid:                      CESM f09_g16 resolution
    parent_variant_label:      r1i1p1f1
    data_specs_version:        01.00.29
    intake_esm_varname:        mrsol
    ...                        ...
    parent_activity_id:        CMIP
    parent_mip_era:            CMIP6
    variable_id:               mrsol
    forcing_index:             1
    realization_index:         1
    intake_esm_dataset_key:    CMIP.SAM0-UNICON.historical.Emon.gn

### 2. Split and Save New Data Files

In [25]:
nc_out = split_and_save(ds_dict)

File with path: ../../data/CMIP6/historical/raw/mrsol/CMIP.SAM0-UNICON.historical.mrsol.nc saved
File with path: ../../data/CMIP6/historical/raw/evspsbl/CMIP.SAM0-UNICON.historical.evspsbl.nc saved
File with path: ../../data/CMIP6/historical/raw/hurs/CMIP.SAM0-UNICON.historical.hurs.nc saved
File with path: ../../data/CMIP6/historical/raw/huss/CMIP.SAM0-UNICON.historical.huss.nc saved
File with path: ../../data/CMIP6/historical/raw/pr/CMIP.SAM0-UNICON.historical.pr.nc saved
File with path: ../../data/CMIP6/historical/raw/evspsblsoi/CMIP.SAM0-UNICON.historical.evspsblsoi.nc saved
File with path: ../../data/CMIP6/historical/raw/evspsblveg/CMIP.SAM0-UNICON.historical.evspsblveg.nc saved
File with path: ../../data/CMIP6/historical/raw/gpp/CMIP.SAM0-UNICON.historical.gpp.nc saved
File with path: ../../data/CMIP6/historical/raw/lai/CMIP.SAM0-UNICON.historical.lai.nc saved
File with path: ../../data/CMIP6/historical/raw/mrro/CMIP.SAM0-UNICON.historical.mrro.nc saved
File with path: ../../data

In [10]:
# ========= Have a look into the new data =======
var = 'mrsol'
open_dataset(os.path.join(savepath,var, f'CMIP.{ds_dict[list(ds_dict.keys())[0]].source_id}.{ds_dict[list(ds_dict.keys())[0]].experiment_id}.{var}.nc'))

<xarray.Dataset>
Dimensions:  (time: 1980, depth: 4, lat: 144, lon: 192)
Coordinates:
  * time     (time) object 1850-01-16 00:00:00 ... 2014-12-16 00:00:00
  * depth    (depth) float64 0.05 0.225 0.675 2.0
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
Data variables:
    mrsol    (time, depth, lat, lon) float32 ...
Attributes: (12/47)
    forcing_index:           2
    source:                  UKESM1.0-LL (2018): \naerosol: UKCA-GLOMAP-mode\...
    experiment_id:           historical
    grid:                    Native N96 grid; 192 x 144 longitude/latitude
    activity_id:             CMIP
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.MOHC.UKESM1...
    ...                      ...
    parent_source_id:        UKESM1-0-LL
    title:                   UKESM1-0-LL output prepared for CMIP6
    physics_index:           1
    variable_id:             mrsol
    license:                 CMIP6 model data produced by the Met Office Hadl...
    intake_esm_dataset_key:  CMIP.UKESM1-0-LL.historical.Emon.gn